In [13]:
import pandas as pd
import numpy as np
import json
from pykrige.ok import OrdinaryKriging
import math

def mercator_projection(lat, lon):
    """ Convert lat, lon in degrees to Mercator projection """
    x = lon
    y = math.log(math.tan((90 + lat) * math.pi / 360)) / (math.pi / 180)
    y = y * 20037508.34 / 180
    return x, y

def generate_kriging_json_mercator(filename, num_points, output_file):
    data = pd.read_csv(filename)
    data.dropna(subset=['Value', 'lat', 'lng'], inplace=True)

    # Convert coordinates to Mercator
    mercator_coords = data.apply(lambda row: mercator_projection(row['lat'], row['lng']), axis=1)
    data['mercator_x'] = mercator_coords.apply(lambda x: x[0])
    data['mercator_y'] = mercator_coords.apply(lambda x: x[1])

    # Define grid in Mercator coordinates
    grid_x = np.linspace(data['mercator_x'].min(), data['mercator_x'].max(), num_points)
    grid_y = np.linspace(data['mercator_y'].min(), data['mercator_y'].max(), num_points)
    grid_x, grid_y = np.meshgrid(grid_x, grid_y)

    # Initialize and execute Ordinary Kriging
    ok_model = OrdinaryKriging(
        data['mercator_x'].values, data['mercator_y'].values, data['Value'].values,
        variogram_model='exponential',
        variogram_parameters={'sill': 94.41677613785475, 'range': 0.1, 'nugget': 13.881621490664376},
        verbose=True,
        enable_plotting=False
    )
    z, ss = ok_model.execute('grid', grid_x[0], grid_y[:,0])

    # Convert grid back to lat, lon for output (optional)
    output = []
    for i in range(num_points):
        for j in range(num_points):
            lon, lat = mercator_projection(grid_y[i, j], grid_x[i, j])
            output.append({
                "lat": lat,
                "lon": lon,
                "value": z[i, j]
            })

    with open(output_file, 'w') as f:
        json.dump(output, f, indent=4)
    
    print(f"Data saved to {output_file}")

# Specify the output filename here
generate_kriging_json_mercator('processed.csv', 100, 'interp_data_mercator.json')


Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'exponential' Variogram Model
Partial Sill: 80.53515464719038
Full Sill: 94.41677613785475
Range: 0.1
Nugget: 13.881621490664376 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...



TypeError: mercator_projection() got an unexpected keyword argument 'inverse'